##### Copyright 2018, Autores do TensorFlow.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Pontos de verificação de treinamento

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/checkpoint"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver no TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar caderno</a></td>
</table>

A frase "Salvando um modelo do TensorFlow" normalmente significa uma de duas coisas:

1. Pontos de verificação, OU
2. SavedModel.

Os pontos de verificação capturam o valor exato de todos os parâmetros ( `tf.Variable` ) usados por um modelo. Os pontos de verificação não contêm nenhuma descrição do cálculo definido pelo modelo e, portanto, normalmente só são úteis quando o código-fonte que usará os valores de parâmetro salvos está disponível.

O formato SavedModel, por outro lado, inclui uma descrição serializada do cálculo definido pelo modelo, além dos valores de parâmetro (ponto de verificação). Os modelos neste formato são independentes do código-fonte que criou o modelo. Portanto, eles são adequados para implantação por meio do TensorFlow Serving, TensorFlow Lite, TensorFlow.js ou programas em outras linguagens de programação (C, C ++, Java, Go, Rust, C # etc. APIs do TensorFlow).

Este guia cobre APIs para escrever e ler pontos de verificação.

In [0]:
import tensorflow as tf

In [0]:
class Net(tf.keras.Model):
  """A simple linear model."""

  def __init__(self):
    super(Net, self).__init__()
    self.l1 = tf.keras.layers.Dense(5)

  def call(self, x):
    return self.l1(x)

In [0]:
net = Net()

## Salvando de APIs de treinamento `tf.keras`

Consulte o[`tf.keras` sobre como salvar e restaurar](./keras/overview.ipynb#save_and_restore) .

`tf.keras.Model.save_weights` salva um ponto de verificação do TensorFlow. 

In [0]:
net.save_weights('easy_checkpoint')

## Escrevendo pontos de verificação


O estado persistente de um modelo do TensorFlow é armazenado em objetos `tf.Variable` Eles podem ser construídos diretamente, mas geralmente são criados por meio de APIs de alto nível, como `tf.keras.layers` ou `tf.keras.Model` .

A maneira mais fácil de gerenciar variáveis é anexando-as a objetos Python e, em seguida, referenciando esses objetos.

As subclasses de `tf.train.Checkpoint` , `tf.keras.layers.Layer` e `tf.keras.Model` rastreiam automaticamente as variáveis atribuídas a seus atributos. O exemplo a seguir constrói um modelo linear simples e, em seguida, grava pontos de verificação que contêm valores para todas as variáveis do modelo.

Você pode facilmente salvar um ponto de verificação de modelo com `Model.save_weights`

### Ponto de verificação manual

#### Configurar

Para ajudar a demonstrar todos os recursos de `tf.train.Checkpoint` defina um conjunto de dados de brinquedo e uma etapa de otimização:

In [0]:
def toy_dataset():
  inputs = tf.range(10.)[:, None]
  labels = inputs * 5. + tf.range(5.)[None, :]
  return tf.data.Dataset.from_tensor_slices(
    dict(x=inputs, y=labels)).repeat().batch(2)

In [0]:
def train_step(net, example, optimizer):
  """Trains `net` on `example` using `optimizer`."""
  with tf.GradientTape() as tape:
    output = net(example['x'])
    loss = tf.reduce_mean(tf.abs(output - example['y']))
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return loss

#### Crie os objetos de checkpoint

Para fazer um ponto de verificação manualmente, você precisará de um objeto `tf.train.Checkpoint` Onde os objetos que você deseja marcar são definidos como atributos no objeto.

Um `tf.train.CheckpointManager` também pode ser útil para gerenciar vários pontos de verificação.

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
dataset = toy_dataset()
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

#### Treinar e verificar o modelo

O seguinte loop de treinamento cria uma instância do modelo e de um otimizador e os reúne em um objeto `tf.train.Checkpoint` Ele chama a etapa de treinamento em um loop em cada lote de dados e grava pontos de verificação no disco periodicamente.

In [0]:
def train_and_checkpoint(net, manager):
  ckpt.restore(manager.latest_checkpoint)
  if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))
  else:
    print("Initializing from scratch.")

  for _ in range(50):
    example = next(iterator)
    loss = train_step(net, example, opt)
    ckpt.step.assign_add(1)
    if int(ckpt.step) % 10 == 0:
      save_path = manager.save()
      print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
      print("loss {:1.2f}".format(loss.numpy()))

In [0]:
train_and_checkpoint(net, manager)

#### Restaurar e continuar o treinamento

Depois do primeiro, você pode passar por um novo modelo e gerente, mas retomar o treinamento exatamente de onde parou:

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
net = Net()
dataset = toy_dataset()
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

train_and_checkpoint(net, manager)

O `tf.train.CheckpointManager` exclui pontos de verificação antigos. Acima está configurado para manter apenas os três pontos de verificação mais recentes.

In [0]:
print(manager.checkpoints)  # List the three remaining checkpoints

Esses caminhos, por exemplo, `'./tf_ckpts/ckpt-10'` , não são arquivos no disco. Em vez disso, eles são prefixos para um `index` e um ou mais arquivos de dados que contêm os valores das variáveis. Esses prefixos são agrupados em um único arquivo de `checkpoint` `'./tf_ckpts/checkpoint'` ) onde o `CheckpointManager` salva seu estado.

In [0]:
!ls ./tf_ckpts

<a id="loading_mechanics"></a>

## Carregando mecânica

O TensorFlow combina variáveis com valores de checkpoint, percorrendo um gráfico direcionado com bordas nomeadas, começando pelo objeto que está sendo carregado. Os nomes de borda normalmente vêm de nomes de atributos em objetos, por exemplo, o `"l1"` em `self.l1 = tf.keras.layers.Dense(5)` . `tf.train.Checkpoint` usa seus nomes de argumento de palavra-chave, como na `"step"` em `tf.train.Checkpoint(step=...)` .

O gráfico de dependência do exemplo acima se parece com isto:

![Visualização do gráfico de dependência para o exemplo de loop de treinamento](https://tensorflow.org/images/guide/whole_checkpoint.svg)

Com o otimizador em vermelho, as variáveis regulares em azul e as variáveis de slot do otimizador em laranja. Os outros nós, por exemplo representando o `tf.train.Checkpoint` , são pretos.

As variáveis de slot fazem parte do estado do otimizador, mas são criadas para uma variável específica. Por exemplo, as `'m'` acima correspondem ao momento, que o otimizador Adam rastreia para cada variável. As variáveis de slot são salvas em um ponto de verificação apenas se a variável e o otimizador forem salvos, portanto, as bordas tracejadas.

Chamar `restore()` em um `tf.train.Checkpoint` enfileira as restaurações solicitadas, restaurando valores de variáveis assim que houver um caminho correspondente do objeto `Checkpoint` Por exemplo, podemos carregar apenas o viés do modelo que definimos acima, reconstruindo um caminho para ele através da rede e da camada.

In [0]:
to_restore = tf.Variable(tf.zeros([5]))
print(to_restore.numpy())  # All zeros
fake_layer = tf.train.Checkpoint(bias=to_restore)
fake_net = tf.train.Checkpoint(l1=fake_layer)
new_root = tf.train.Checkpoint(net=fake_net)
status = new_root.restore(tf.train.latest_checkpoint('./tf_ckpts/'))
print(to_restore.numpy())  # We get the restored value now

O gráfico de dependência para esses novos objetos é um subgráfico muito menor do ponto de verificação maior que escrevemos acima. Inclui apenas a polarização e um contador de salvamento que `tf.train.Checkpoint` usa para numerar os pontos de verificação.

![Visualização de um subgráfico para a variável de polarização](https://tensorflow.org/images/guide/partial_checkpoint.svg)

`restore()` retorna um objeto de status, que possui asserções opcionais. Todos os objetos que criamos em nosso novo `Checkpoint` foram restaurados, então `status.assert_existing_objects_matched()` passa.

In [0]:
status.assert_existing_objects_matched()

Existem muitos objetos no ponto de verificação que não corresponderam, incluindo o kernel da camada e as variáveis do otimizador. `status.assert_consumed()` só passa se o ponto de verificação e o programa correspondem exatamente, e lançaria uma exceção aqui.

### Restaurações atrasadas

`Layer` no TensorFlow podem atrasar a criação de variáveis para sua primeira chamada, quando as formas de entrada estão disponíveis. Por exemplo, a forma `Dense` depende das formas de entrada e saída da camada e, portanto, a forma de saída exigida como argumento do construtor não é informação suficiente para criar a variável por conta própria. Visto que chamar uma `Layer` também lê o valor da variável, uma restauração deve acontecer entre a criação da variável e seu primeiro uso.

Para oferecer suporte a esse idioma, `tf.train.Checkpoint` enfileira restaurações que ainda não têm uma variável correspondente.

In [0]:
delayed_restore = tf.Variable(tf.zeros([1, 5]))
print(delayed_restore.numpy())  # Not restored; still zeros
fake_layer.kernel = delayed_restore
print(delayed_restore.numpy())  # Restored

### Inspecionando manualmente os pontos de verificação

`tf.train.list_variables` lista as chaves de checkpoint e formas de variáveis em um checkpoint. As chaves de ponto de verificação são caminhos no gráfico exibido acima.

In [0]:
tf.train.list_variables(tf.train.latest_checkpoint('./tf_ckpts/'))

### Rastreamento de lista e dicionário

Assim como acontece com as atribuições diretas de atributos, como `self.l1 = tf.keras.layers.Dense(5)` , a atribuição de listas e dicionários aos atributos rastreará seus conteúdos.

In [0]:
save = tf.train.Checkpoint()
save.listed = [tf.Variable(1.)]
save.listed.append(tf.Variable(2.))
save.mapped = {'one': save.listed[0]}
save.mapped['two'] = save.listed[1]
save_path = save.save('./tf_list_example')

restore = tf.train.Checkpoint()
v2 = tf.Variable(0.)
assert 0. == v2.numpy()  # Not restored yet
restore.mapped = {'two': v2}
restore.restore(save_path)
assert 2. == v2.numpy()

Você pode notar objetos de invólucro para listas e dicionários. Esses wrappers são versões verificáveis das estruturas de dados subjacentes. Assim como o carregamento baseado em atributo, esses wrappers restauram o valor de uma variável assim que ela é adicionada ao contêiner.

In [0]:
restore.listed = []
print(restore.listed)  # ListWrapper([])
v1 = tf.Variable(0.)
restore.listed.append(v1)  # Restores v1, from restore() in the previous cell
assert 1. == v1.numpy()

O mesmo rastreamento é aplicado automaticamente às subclasses de `tf.keras.Model` e pode ser usado, por exemplo, para rastrear listas de camadas.

## Salvando pontos de verificação baseados em objetos com o Estimator

Veja o [guia](https://www.tensorflow.org/guide/estimator) do Estimador.

Os estimadores, por padrão, salvam pontos de verificação com nomes de variáveis em vez do gráfico de objeto descrito nas seções anteriores. `tf.train.Checkpoint` aceitará pontos de verificação baseados em nomes, mas os nomes das variáveis podem mudar ao mover partes de um modelo para fora do `model_fn` do Estimador. Salvar pontos de verificação baseados em objetos torna mais fácil treinar um modelo dentro de um Estimador e, em seguida, usá-lo fora de um.

In [0]:
import tensorflow.compat.v1 as tf_compat

In [0]:
def model_fn(features, labels, mode):
  net = Net()
  opt = tf.keras.optimizers.Adam(0.1)
  ckpt = tf.train.Checkpoint(step=tf_compat.train.get_global_step(),
                             optimizer=opt, net=net)
  with tf.GradientTape() as tape:
    output = net(features['x'])
    loss = tf.reduce_mean(tf.abs(output - features['y']))
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  return tf.estimator.EstimatorSpec(
    mode,
    loss=loss,
    train_op=tf.group(opt.apply_gradients(zip(gradients, variables)),
                      ckpt.step.assign_add(1)),
    # Tell the Estimator to save "ckpt" in an object-based format.
    scaffold=tf_compat.train.Scaffold(saver=ckpt))

tf.keras.backend.clear_session()
est = tf.estimator.Estimator(model_fn, './tf_estimator_example/')
est.train(toy_dataset, steps=10)

`tf.train.Checkpoint` pode então carregar os pontos de verificação do Estimator de seu `model_dir` .

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
net = Net()
ckpt = tf.train.Checkpoint(
  step=tf.Variable(1, dtype=tf.int64), optimizer=opt, net=net)
ckpt.restore(tf.train.latest_checkpoint('./tf_estimator_example/'))
ckpt.step.numpy()  # From est.train(..., steps=10)

## Resumo

Os objetos do TensorFlow fornecem um mecanismo automático fácil para salvar e restaurar os valores das variáveis que usam.
